In [1]:
import os
import pandas as pd
from typing import List, Tuple
import numpy as np
import json

In [2]:
DATASET_PATH = "../dataset"
INDEX_FIELD = "timestamp"
DATA_FIELD = "num_request"

PROACTIVE_RESULT_ROOT="proactive_result"

In [3]:
def get_data_file_list(dataset_path: str) -> List[str]:
    return os.listdir(dataset_path)

In [4]:
def read_dataset(csv_path: str,index_field:str,data_field:str) -> Tuple[np.ndarray, np.ndarray]:
    df = pd.read_csv(csv_path)
    return df[index_field].to_numpy(), df[data_field].to_numpy()

In [5]:
def read_candidate_cpds(path: str) -> List[int]:
    candidate_cpds = None
    with open(path, "r") as f:
        candidate_cpds = json.load(f)
    return candidate_cpds

In [6]:
class Simulator:
    def __init__(self, workload: np.ndarray, capacity_per_pod: float, sla: float, fluctuation: float, t_cooldown: int = 1) -> None:
        self.t = 0
        self.sla = sla
        self.upper_sla = sla+fluctuation
        self.lower_sla = sla-fluctuation
        self.workload = workload
        self.capaciy_per_pod = capacity_per_pod
        self.t_cooldown = t_cooldown
        self.r_cpu_list = []
        self.n_pod_list = []
        self.n_scale = 0
        self.workload_list = []

    def run(self):
        n_pod = int(np.ceil(self.workload[0]/(self.sla*self.capaciy_per_pod)))
        t_last_scale = -1
        while self.t < len(self.workload)-2:
            current_workload = self.workload[self.t]
            self.workload_list.append(current_workload)
            future_workload = np.average([self.workload[self.t+1], self.workload[self.t+2]])
            if self.t-t_last_scale > self.t_cooldown:
                target_n_pod = int(np.ceil(future_workload/(self.sla*self.capaciy_per_pod)))
                if target_n_pod != n_pod:
                    n_pod = target_n_pod
                    t_last_scale = self.t
                    self.n_scale += 1
            r_cpu = 1.0*current_workload/(n_pod*self.capaciy_per_pod)
            self.r_cpu_list.append(r_cpu)
            self.n_pod_list.append(n_pod)
            self.t += 1
    
    def r_sla_violate(self):
        return 1.0*(len(np.where(np.array(self.r_cpu_list) > self.upper_sla)[0])+len(np.where(np.array(self.r_cpu_list) < self.lower_sla)[0]))/len(self.r_cpu_list)
    
    def r_sla_violate_upper(self):
        return 1.0*(len(np.where(np.array(self.r_cpu_list) > self.upper_sla)[0]))/len(self.r_cpu_list)
    
    def r_sla_violate_lower(self):
        return 1.0*(len(np.where(np.array(self.r_cpu_list) < self.lower_sla)[0]))/len(self.r_cpu_list)

In [7]:
def dump_dir(dir: str, path: str):
    with open(path, "w") as f:
        json.dump(dir, f, indent=4)

In [8]:
workload_to_skip_list = ["workload_1998-06-13", "workload_1998-06-14", "workload_1998-06-20", "workload_1998-06-21", "workload_1998-06-27", "workload_1998-06-28","workload_1998-07-04"]

In [9]:
if not os.path.exists(PROACTIVE_RESULT_ROOT):
    os.makedirs(PROACTIVE_RESULT_ROOT)
data_file_list = get_data_file_list(DATASET_PATH)
x, y = [], []
n_scale_dict = {}
r_cpu_dict = {}
n_pod_dict = {}
workload_dict = {}
r_sla_vio_dict={}
r_sla_vio_upper_dict={}
r_sla_vio_lower_dict={}
for file_name in data_file_list:
    workload_name = file_name.split(".")[0]
    if workload_name in workload_to_skip_list:
        continue
    print("read %s" % (file_name))
    np_index, np_data = read_dataset(os.path.join(DATASET_PATH, file_name), INDEX_FIELD, DATA_FIELD)
    np_data = np.floor(np_data/10.0)
    simulator = Simulator(workload=np_data, capacity_per_pod=2500, sla=0.5, fluctuation=0.1, t_cooldown=1)
    simulator.run()
    n_scale_dict[workload_name] = simulator.n_scale
    r_cpu_dict[workload_name] = simulator.r_cpu_list
    n_pod_dict[workload_name] = simulator.n_pod_list
    workload_dict[workload_name] = simulator.workload_list
    r_sla_vio_dict[workload_name]=simulator.r_sla_violate()
    r_sla_vio_upper_dict[workload_name]=simulator.r_sla_violate_upper()
    r_sla_vio_lower_dict[workload_name]=simulator.r_sla_violate_lower()
dump_dir(n_scale_dict, os.path.join(PROACTIVE_RESULT_ROOT, "n_scale.json"))
dump_dir(r_cpu_dict, os.path.join(PROACTIVE_RESULT_ROOT, "r_cpu.json"))
dump_dir(n_pod_dict, os.path.join(PROACTIVE_RESULT_ROOT, "n_pod.json"))
dump_dir(workload_dict, os.path.join(PROACTIVE_RESULT_ROOT, "workload.json"))
dump_dir(r_sla_vio_dict, os.path.join(PROACTIVE_RESULT_ROOT, "r_sla_violate.json"))
dump_dir(r_sla_vio_upper_dict, os.path.join(PROACTIVE_RESULT_ROOT, "r_sla_violate_upper.json"))
dump_dir(r_sla_vio_lower_dict, os.path.join(PROACTIVE_RESULT_ROOT, "r_sla_violate_lower.json"))

read workload_1998-06-10.csv
read workload_1998-06-11.csv
read workload_1998-06-12.csv
read workload_1998-06-15.csv
read workload_1998-06-16.csv
read workload_1998-06-17.csv
read workload_1998-06-18.csv
read workload_1998-06-19.csv
read workload_1998-06-22.csv
read workload_1998-06-23.csv
read workload_1998-06-24.csv
read workload_1998-06-25.csv
read workload_1998-06-26.csv
read workload_1998-06-29.csv
read workload_1998-06-30.csv
read workload_1998-07-03.csv
read workload_1998-07-07.csv
read workload_1998-07-08.csv
